<a href="https://colab.research.google.com/github/RaunaqJahan/mobile-website/blob/main/raunaq47.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import zipfile
import os
import shutil
import glob

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
Zip_ref = zipfile.ZipFile("/content/drive/MyDrive/dataset.zip","r")
Zip_ref.extractall("/tmp")
Zip_ref.close()

In [8]:
base_dir ="/tmp/dataset"
print(os.listdir("/tmp/dataset"))

['diseased', 'healthy']


In [9]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.layers import Dense

In [10]:
IMAGE_SIZE=1200
BATCH_SIZE=32
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1
    )

test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
     rescale=1./255,
     validation_split=0.1
)

train_datagen=train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
)

test_datagen=test_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 135 images belonging to 2 classes.
Found 14 images belonging to 2 classes.


In [19]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=64,padding='same',strides=2,kernel_size=3,activation='relu',input_shape=(1200,1200,3)))
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

model.add(tf.keras.layers.Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(2,activation='softmax'))

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 600, 600, 64)      1792      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 300, 300, 64)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 150, 150, 32)      18464     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 180000)            0         
                                                                 
 dense_2 (Dense)             (None, 2)                

In [21]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])


In [22]:
history = model.fit(train_datagen,epochs=10,validation_data=test_datagen)

Epoch 1/10
5/5 [==============================] - 44s 10s/step - loss: 2.3341 - accuracy: 0.5630 - val_loss: 0.5771 - val_accuracy: 0.7143
Epoch 2/10
5/5 [==============================] - 41s 10s/step - loss: 0.6933 - accuracy: 0.5259 - val_loss: 0.6978 - val_accuracy: 0.3571
Epoch 3/10
5/5 [==============================] - 42s 8s/step - loss: 0.6614 - accuracy: 0.6593 - val_loss: 0.6545 - val_accuracy: 0.7143
Epoch 4/10
5/5 [==============================] - 43s 10s/step - loss: 0.5898 - accuracy: 0.7111 - val_loss: 0.5560 - val_accuracy: 0.7143
Epoch 5/10
5/5 [==============================] - 42s 10s/step - loss: 0.5574 - accuracy: 0.6889 - val_loss: 0.5450 - val_accuracy: 0.7143
Epoch 6/10
5/5 [==============================] - 41s 8s/step - loss: 0.5279 - accuracy: 0.7037 - val_loss: 0.5590 - val_accuracy: 0.7143
Epoch 7/10
5/5 [==============================] - 42s 10s/step - loss: 0.5145 - accuracy: 0.7481 - val_loss: 0.6400 - val_accuracy: 0.6429
Epoch 8/10
5/5 [=============

In [23]:
model.save('cnn_model.h5')

In [24]:
from keras.models import load_model

model1 = load_model('/content/cnn_model.h5',compile=False)

In [38]:
lab = train_datagen.class_indices
print(lab)
lab={k:v for v,k in lab.items()}
print(lab)

{'diseased': 0, 'healthy': 1}
{0: 'diseased', 1: 'healthy'}


# New Section

In [39]:
from keras.utils import image_utils
def output(location):
    img=image_utils.load_img(location,target_size=(1200,1200,3))
    img=image_utils.img_to_array(img)
    img=img/255
    img=np.expand_dims(img,[0])
    answer=model1.predict(img)
    y_class = answer.argmax(axis=-1)
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res = lab[y]
    return res

In [43]:
import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

Saving dis.jpeg to dis (1).jpeg


In [46]:
img='/content/dis.jpeg'
output(img)

1/1 [==============================] - 0s 24ms/step


'diseased'